# Completion


# Clarifications


# Questions to Ask



In [1]:
from utils import *

import nest_asyncio
import asyncio

# Only run nest_asyncio in a Jupyter Notebook environment
nest_asyncio.apply()

In [2]:
# Load the respective files

async def get_all_dataframes():
    return await asyncio.gather(
        load_intake24(),
        load_heifa_ingredients(),
        load_heifa_recipes(),
        load_heifa_scores()
    )

intake24_df, heifa_food_df, heifa_recipes_df, heifa_scores_df = asyncio.run(get_all_dataframes())

Before dropping function: 5754 rows
After dropping function: 5670 rows


c:\Users\Tazeek\Desktop\Projects\autotake-food24\utils.py:81: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  heifa_recipes_df = pd.read_csv('files/heifa_recipes.csv')


# Breakdown of Intake 24:

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [3]:
user_dict = create_user_objects(intake24_df)

#for user_id in user_dict.keys():

#    print(f"Printing for User {user_id}")
#    user_obj = user_dict[user_id]
#    user_obj.print_information()

# Breakdown of HEIFA (Food Composition)

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [4]:
# Create the objects
food_composition_dict = create_food_objects(heifa_food_df)

#for key, food_comp_obj in food_composition_dict.items():
#    food_comp_obj.print_full_details()

# Breakdown of HEIFA (Recipes)

- Every recipe has multiple ingredients
- Keys are repeated across rows (similar to Survey ID of Intake24)
- Every ingredient has respective proportion to the recipe

In [5]:
recipe_dict = create_recipe_objects(heifa_recipes_df)

#for id, recipe_obj in recipe_dict.items():
#    print(f"Printing for ID {id}\n")
#    recipe_obj.print_ingredients_information()

## Mapping between Intake24 and HEIFA Ingredients

- For each user, extract the given nutrients and store in an array.
- This is from ALL the survey data.
- We don't care about the order here.
- The array will contain a list of dictionaries/JSON.

In the array:

- Use the HEIFA ID (from user) to map to the HEIFA Ingredients' HEIFA ID.
- Check if a result is found or not.
- Check if it requires a recipe or not.

## Mapping between Intake24 and HEIFA Recipes

This is in case a recipe is found (The second step).

- For the given recipe, extract the given nutrients ID and proportion, store in an array.
- We don't care about the order here.
- The array will contain a list of dictionaries.

In the array:

- Use the HEIFA ID (from the recipes) to map the HEIFA Ingredients' HEIFA ID.
- Check the energy and serving size.

In [6]:
user_daily_intake = calculate_user_servings(user_dict, food_composition_dict, recipe_dict)


HEIFA ID 8416 not found

HEIFA ID 8416 not found

HEIFA ID 6061 not found

HEIFA ID 8416 not found

HEIFA ID 8416 not found

HEIFA ID 8416 not found


# Test with Samara's CSV file and post the updates here

**Assumption**: This should be the same as Intake24 file format.

## Errors encountered

**Column  names between Intake24 and Latrobe**
- "Start date (AEST)" -> Different from Intake24 (used 'Start Time'). -> "RESOLVED"
- 'Nutrient table code (original)' -> Different from Intake24 (used 'Nutrient table code'). -> "RESOLVED"
- 'Energy, with dietary fibre (kJ)' -> Different from Intake24 (used 'Energy, with dietary fibre'). -> "RESOLVED"

**Nutrient ID related**
- Values of "N/A" in the Nutrient ID still present. -> "RESOLVED"
- Unknown codes still present (8416) -> "RESOLVED"
- Row difference before and after dropping: 6028 vs 5613 (415) -> "RESOLVED"

**Inside the file**:
- Some nutrient ID is the food description and not the ID (Example: Porridge, made with light milk) -> "RESOLVED"
- Nutrient ID does not have the ID from Row 3072 to 3294; it has description (same as previous reason) -> "RESOLVED"
- Some values of energy are not found; they are shown as #VALUE! (#VALUE! present in the google sheet) -> "RESOLVED"


In [7]:
from utils import *

latrobe_df = load_latrobe_file()

Before dropping function: 6114 rows
After dropping function: 6111 rows


In [8]:
user_latrobe_dict = create_user_objects(latrobe_df)

for user_id in user_latrobe_dict.keys():

    print(f"Printing for User {user_id}")
    user_obj = user_latrobe_dict[user_id]
    user_obj.print_information()

Printing for User 1
Printing for Survey ID 5d3e72da-a241-4e8e-837b-021accf9a37c on date 03/08/2022

HEIFA code(s) for Breakfast: ['04B10082', '02B10604', '09A10204', '01B10301']

Details for 04B10082:
Portion size of 15.5g/ml
Energy (Dietary Fibre included): 357.90 kJ

Details for 02B10604:
Portion size of 54.0g/ml
Energy (Dietary Fibre included): 638.28 kJ

Details for 09A10204:
Portion size of 16.42g/ml
Energy (Dietary Fibre included): 32.51 kJ

Details for 01B10301:
Portion size of 194.92g/ml
Energy (Dietary Fibre included): 11.70 kJ

HEIFA code(s) for Lunch: ['02B10775', '09A10204', '01B10301', '01B20399']

Details for 02B10775:
Portion size of 190.0g/ml
Energy (Dietary Fibre included): 1993.10 kJ

Details for 09A10204:
Portion size of 16.42g/ml
Energy (Dietary Fibre included): 32.51 kJ

Details for 01B10301:
Portion size of 195.2g/ml
Energy (Dietary Fibre included): 11.71 kJ

Details for 01B20399:
Portion size of 375.0g/ml
Energy (Dietary Fibre included): 0.00 kJ

HEIFA code(s) fo

In [9]:
latrobe_user_daily_intake = calculate_user_servings(user_latrobe_dict, food_composition_dict, recipe_dict)

In [10]:
# Create the HEIFA scores list
heifa_scores_dict = create_scores_objects(heifa_scores_df)
latrobe_user_heifa_scores = calculate_heifa_scores(heifa_scores_dict, latrobe_user_daily_intake)

VEGGIE!
{'Green': 0.33, 'Red and orange': 2.3600000000000003, 'Tuber and bulb': 2.48}
2

VEGGIE!
{'Green': 0.54, 'Red and orange': 0.9400000000000001, 'Tuber and bulb': 0.31}
0

VEGGIE!
{'Tuber and bulb': 1.96, 'Green': 0.15, 'Other': 0.41, 'Red and orange': 0.8700000000000001}
1

FRUIT!
{'Tropical and subtropical': 1.39}
0

VEGGIE!
{'Cruciferous': 1.1800000000000002, 'Green': 2.0, 'Red and orange': 1.9600000000000002, 'Other': 1.4300000000000002, 'Tuber and bulb': 1.42}
5

VEGGIE!
{'Cruciferous': 1.1800000000000002, 'Green': 1.97, 'Red and orange': 1.9600000000000002, 'Other': 1.4300000000000002, 'Tuber and bulb': 1.42}
5

VEGGIE!
{'Tuber and bulb': 0.87, 'Green': 0.6900000000000002, 'Cruciferous': 0.35, 'Other': 0.41000000000000003, 'Red and orange': 1.15}
1

FRUIT!
{'Berry': 0.13, 'Pome': 0.57}
0

VEGGIE!
{'Red and orange': 3.95, 'Tuber and bulb': 1.9000000000000001, 'Green': 1.54, 'Cruciferous': 0.77, 'Other': 0.9100000000000001}
3

FRUIT!
{'Berry': 0.19}
0

VEGGIE!
{'Other': 1.160

In [11]:
# Display
for user_id, daily_intake_dict in latrobe_user_daily_intake.items():

    for date, food_group_dict in daily_intake_dict.items():
        print(f"Breakdown of User {user_id} on {date}:")

        individual_dict = food_group_dict['individual']
        total_dict = food_group_dict['total']

        # Sort so can display in alphabetical order
        individual_dict = dict(sorted(individual_dict.items()))

        for food_group, total_serving in individual_dict.items():
            print(f"- {food_group}: {total_serving:.2f} serves")

        print("")
        
        print("***HEIFA SCORES CONVERSION (START)***\n")
        total_dict = dict(sorted(total_dict.items()))

        for food_group, total_serving in total_dict.items():
            
            print(f"> {food_group}: {total_serving:.2f} serves")

            if food_group not in latrobe_user_heifa_scores[user_id][date]:
                print("* No score")
                print("")
                continue

            gender_scores = latrobe_user_heifa_scores[user_id][date][food_group]

            male_score = gender_scores['male_score']
            female_score = gender_scores['female_score']
        
            print(f"* Male score: {male_score}")
            print(f"* Female score: {female_score}")
            print("")
        
        print("***HEIFA SCORES CONVERSION (END)***")
        print("")
        print("=" * 20)

Breakdown of User 1 on 5d3e72da-a241-4e8e-837b-021accf9a37c:
- Dairy and alternatives: 0.63 serves
- Discretionary: 5.26 serves
- Grains and cereals/Non-wholegrains: 4.79 serves
- Meat and alternatives: 1.79 serves
- No food group: 0.00 serves

***HEIFA SCORES CONVERSION (START)***

> Dairy and alternatives: 0.63 serves
* Male score: 2.0
* Female score: 2.0

> Discretionary: 5.26 serves
* Male score: 2.5
* Female score: 2.5

> Grains and cereals: 4.79 serves
* Male score: 3.34
* Female score: 3.34

> Meat and alternatives: 1.79 serves
* Male score: 4.0
* Female score: 6.0

> No food group: 0.00 serves
* No score

***HEIFA SCORES CONVERSION (END)***

Breakdown of User 1 on 335596f7-b09f-4052-8bc0-da642c4f615a:
- Dairy and alternatives: 2.33 serves
- Discretionary: 9.04 serves
- Grains and cereals/Non-wholegrains: 6.23 serves
- Meat and alternatives: 2.06 serves
- No food group: 0.00 serves
- Vegetables/Green: 0.33 serves
- Vegetables/Red and orange: 2.36 serves
- Vegetables/Tuber and bu

# Calculating the HEIFA Scores

Heifa scores are to be calculated on a **daily basis**.

To calculate them, let's break them down:

- Break down by user
- Break down by date
- Break down by major food group (Example: Vegetables/Green -> Vegetables is the major food group)
- Break down by sub-food group of the major (Example: Vegetables/Green -> Green is the sub-food group)
- Compare the scores by gender (male and female)

There are some exceptions to the rule, based on the HEIFA scores guideline:

- Grains and cereals/Wholegrains -> This is to be calculated separately as "Grains and cereals" and "Wholegrains".

Varieties:

- Store the variety score in a different dictionary
- Find the score of each variety group separately (based on the HEIFA rule book)

In [12]:
user_daily_intake

{1: {'5d3e72da-a241-4e8e-837b-021accf9a37c': {'individual': {'Discretionary': 3.33,
    'Grains and cereals/Non-wholegrains': 2.56,
    'Dairy and alternatives': 0.18,
    'No food group': 0,
    'Vegetables/Tuber and bulb': 0.19,
    'Vegetables/Red and orange': 0.39,
    'Meat and alternatives': 0.59,
    'Water': 0,
    'Vegetables/Other': 0.25,
    'Vegetables/Green': 0.4,
    'Vegetables/Cruciferous': 0.26},
   'total': {'Discretionary': 3.33,
    'Grains and cereals': 2.56,
    'Dairy and alternatives': 0.18,
    'No food group': 0,
    'Vegetables': 1.49,
    'Meat and alternatives': 0.59,
    'Water': 0},
   'variations': {'Grains and cereals': {'Non-wholegrains': 2.56},
    'Vegetables': {'Tuber and bulb': 0.19,
     'Red and orange': 0.39,
     'Other': 0.25,
     'Green': 0.4,
     'Cruciferous': 0.26}}},
  '335596f7-b09f-4052-8bc0-da642c4f615a': {'individual': {'Discretionary': 6.199999999999999,
    'Grains and cereals/Non-wholegrains': 2.44,
    'Dairy and alternatives': 

In [13]:
# Create the HEIFA scores list
heifa_scores_dict = create_scores_objects(heifa_scores_df)

user_heifa_scores = calculate_heifa_scores(heifa_scores_dict, user_daily_intake)

VEGGIE!
{'Tuber and bulb': 0.19, 'Red and orange': 0.39, 'Other': 0.25, 'Green': 0.4, 'Cruciferous': 0.26}
0

VEGGIE!
{'Tuber and bulb': 2.48}
1

VEGGIE!
{'Green': 0.54, 'Red and orange': 0.9400000000000001, 'Tuber and bulb': 0.31}
0

VEGGIE!
{'Tuber and bulb': 1.91, 'Green': 0.31, 'Red and orange': 0.8200000000000001, 'Other': 0.36}
1

FRUIT!
{'Tropical and subtropical': 1.39}
0

VEGGIE!
{'Tuber and bulb': 0.87, 'Green': 0.7200000000000002, 'Cruciferous': 0.35, 'Other': 0.41000000000000003, 'Red and orange': 1.15}
1

VEGGIE!
{'Cruciferous': 0.83, 'Green': 1.15, 'Red and orange': 0.51, 'Other': 1.02, 'Tuber and bulb': 0.45}
2

FRUIT!
{'Berry': 0.13, 'Pome': 0.57}
0

VEGGIE!
{'Red and orange': 3.95, 'Tuber and bulb': 1.9000000000000001, 'Green': 1.54, 'Cruciferous': 0.77, 'Other': 0.9100000000000001}
3

FRUIT!
{'Berry': 0.19}
0

VEGGIE!
{'Other': 1.1600000000000001, 'Red and orange': 2.78, 'Legumes': 0.39, 'Tuber and bulb': 1.28, 'Green': 1.06, 'Cruciferous': 0.53}
4

FRUIT!
{'Berry': 0

In [15]:
# Display
for user_id, daily_intake_dict in user_daily_intake.items():

    for survey_id, food_group_dict in daily_intake_dict.items():
        print(f"Breakdown of User {user_id} for Survey ID {survey_id}:")

        individual_dict = food_group_dict['individual']
        total_dict = food_group_dict['total']
        variations_dict = food_group_dict['variations']

        # Sort so can display in alphabetical order
        individual_dict = dict(sorted(individual_dict.items()))

        for food_group, total_serving in individual_dict.items():
            print(f"- {food_group}: {total_serving:.2f} serves")

        print("")
        
        print("***HEIFA SCORES CONVERSION (START)***\n")
        total_dict = dict(sorted(total_dict.items()))

        for food_group, total_serving in total_dict.items():
            
            print(f"> {food_group}: {total_serving:.2f} serves")

            if food_group not in user_heifa_scores[user_id][survey_id]:
                print("* No score")
                print("")
                continue

            if food_group in variations_dict:

                variations = variations_dict[food_group]
                for sub_group, serving_size in variations.items():
                    print(f"-- {sub_group}: {serving_size:.2f} serves")

            gender_scores = user_heifa_scores[user_id][survey_id][food_group]

            male_score = gender_scores['male_score']
            female_score = gender_scores['female_score']
        
            print(f"* Male score: {male_score}")
            print(f"* Female score: {female_score}")
            print("")
        
        print("***HEIFA SCORES CONVERSION (END)***")
        print("")
        print("=" * 20)

Breakdown of User 1 for Survey ID 5d3e72da-a241-4e8e-837b-021accf9a37c:
- Dairy and alternatives: 0.18 serves
- Discretionary: 3.33 serves
- Grains and cereals/Non-wholegrains: 2.56 serves
- Meat and alternatives: 0.59 serves
- No food group: 0.00 serves
- Vegetables/Cruciferous: 0.26 serves
- Vegetables/Green: 0.40 serves
- Vegetables/Other: 0.25 serves
- Vegetables/Red and orange: 0.39 serves
- Vegetables/Tuber and bulb: 0.19 serves
- Water: 0.00 serves

***HEIFA SCORES CONVERSION (START)***

> Dairy and alternatives: 0.18 serves
* Male score: 1.0
* Female score: 1.0

> Discretionary: 3.33 serves
* Male score: 7.5
* Female score: 7.5

> Grains and cereals: 2.56 serves
-- Non-wholegrains: 2.56 serves
* Male score: 1.67
* Female score: 1.67

> Meat and alternatives: 0.59 serves
* Male score: 1.0
* Female score: 2.0

> No food group: 0.00 serves
* No score

> Vegetables: 1.49 serves
-- Tuber and bulb: 0.19 serves
-- Red and orange: 0.39 serves
-- Other: 0.25 serves
-- Green: 0.40 serves